In [ ]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import preprocessing

Importando os dados:

In [ ]:
adult_train = pd.read_csv("../input/adult-pmr3508/train_data.csv",index_col=['Id'],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
adult_test = pd.read_csv("../input/adult-pmr3508/test_data.csv",index_col=['Id'],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [ ]:
adult_train.shape

In [ ]:
adult_train.head()

Vamos tratar os dados faltantes, primeiro identificando quais parâmetros apresentam dados faltantes.

In [ ]:
Faltantes = adult_train.isnull().sum().sort_values(ascending = False)
Faltantes.head()

In [ ]:
plt.figure(figsize=(17, 7))
adult_train.workclass.value_counts().plot(kind = 'pie')

Vamos atribuir a moda do parâmetro workclass nos dados faltantes dado que grande parte das pessoas trabalham no setor privado. 

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['native.country'].value_counts().plot(kind = 'pie')

A maior parte das pessoas desse banco de dados são americanas, é sensato retirar esse parâmetro para fazer o treinamento no fim já que esse parâmetro está muito enviesado para pessoas americanas, se quereremos fazer predição, como não sabemos se serão para americanos ou não é útil retirar esse parametro. No entanto, como a maior parte das pessoas desse banco de dados são americanos é conveninente atribuir a nacionalidade americana aos dados faltantes desse parâmetro.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train.occupation.value_counts().plot(kind = 'pie')

Os dados de occupation são bem distribuídos, no entanto vou substituir os dados faltantes pela moda Prof-specialty mas poderia ser por exemplo Craft-repair por não serem estão distantes em número de ocorrência.

Completando com a moda:

In [ ]:
adult_train["workclass"] = adult_train["workclass"].fillna(adult_train["workclass"].describe().top)
adult_train['native.country'] = adult_train['native.country'].fillna(adult_train['native.country'].describe().top);
adult_train['occupation'] = adult_train['occupation'].fillna(adult_train['occupation'].describe().top);

Verificou-se todos os dados faltantes foram eliminados:

In [ ]:
Faltantes = adult_train.isnull().sum().sort_values(ascending = False)
Faltantes.head()

Realizando o mesmo para os dados de teste:

In [ ]:
adult_test["workclass"] = adult_test["workclass"].fillna(adult_test["workclass"].describe().top)
adult_test['native.country'] = adult_test['native.country'].fillna(adult_test['native.country'].describe().top);
adult_test['occupation'] = adult_test['occupation'].fillna(adult_test['occupation'].describe().top);

In [ ]:
Faltantes = adult_test.isnull().sum().sort_values(ascending = False)
Faltantes.head()

Agora, vou verificar como os parâmetros se relacionam com "income", selecionando os parâmetros mais importantes para o modelo final

In [ ]:
plt.figure(figsize=(15, 7))
adult_train.groupby("income").age.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel('Age')
plt.ylabel('quantity')
plt.title('Age histogram')

A maior parte das pessoas mais bem remuneradas estão na faixa dos 40 anos, é de se esperar dado que nessa idade que se atinge uma certa maturidade profissional. Vale observar que é bem possível que essa concentração nessa idade seja maior ainda do que a apresentada dado que grande parte das pessoas do banco de dados estão na faixa dos vinte anos ainda.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['education.num'].value_counts().plot(kind = 'pie')

O parametro education será eliminado dado que education.num representa a mesma informação. A maior parte das pessoas estã no nivel de educação 9, 10 e 13. Poucas pessoas tem qualificação superior a essas, mas também poucas pessoas tem abaixo dessas.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['marital.status'].value_counts().plot(kind = 'pie')

A maior parte dos dados correspondem a pessoas casadas ou não casadas.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['occupation'].value_counts().plot(kind = 'pie')

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['hours.per.week'].value_counts().plot(kind = 'pie')

A maior parte das pessoas consultadas trabalham 40 horas por semana mas algumas passam disso podendo trabalhar até 60 horas.

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['capital.loss'].value_counts().plot(kind = 'pie')

In [ ]:
plt.figure(figsize=(17, 7))
adult_train['capital.loss'].value_counts().plot(kind = 'pie')

Capital loss e gain não aparentam ser dados importantes pois quase ninguém dos dados apresentam esse tipo de investimento. Porém nos testes feitos retirá-los afeta negativamente o resultado final

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").workclass.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel("Workclass")
plt.ylabel('quantity')
plt.title('Workclass histogram')

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").relationship.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel("Relationship Status")
plt.ylabel('Relationship Status')
plt.title('Relationship histogram')

A maior parte das pessoas bem remuneradas são maridos. No entanto esses dados parecem serem incompletos dado que poucas esposas foram consultadas. Porém pode-se aceitar esse resultado dado que posteriormente veremos que há uma desigualdade de gênero presente na remuneração e portanto em relação a estado civil maridos ganhariam mais do que esposas.

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").race.hist()
plt.legend(['<=50k','>50k'])
plt.xlabel("Race Status")
plt.ylabel('Race Status')
plt.title('Race histogram')

É possivel ver que as pessoas mais bem remuneradas desses dados são caucasianas, no entanto quase todas as pessoas consultadas são caucasianas. Seria interessante que esses dados tivessem mais representatividade de outras raças. Irei considerar esse parâmetro, no entanto ele não parece ser aceitável para países com mais diversidade cultural como o próprio Brasil.

In [ ]:
plt.figure(figsize=(20, 7))
adult_train.groupby("income").sex.value_counts().plot(kind = 'pie')
plt.legend(['<=50k','>50k'])
plt.xlabel("Sex Status")
plt.ylabel('Sex Status')
plt.title('Sex histogram')

Como dito, os dados mostram uma desigualdade de gênero presente na sociedade na questão da remuneração. Das pessoas melhor remuneradas, a grande maioria são homens.

Aqui serão transformadas os parâmetros que não são numéricos em numéricos para poderem ser melhor trabalhados.

In [ ]:
adult_train_num=adult_train.apply(preprocessing.LabelEncoder().fit_transform)
adult_test_num=adult_test.apply(preprocessing.LabelEncoder().fit_transform)

In [ ]:
adult_train_num.head()

Construindo então a matriz de correlação entre parâmetros:

In [ ]:
rel = adult_train_num.drop(['income'],axis = 1)
rel_bi = adult_train_num['income']
rel = pd.concat([rel, rel_bi], axis = 1)
plt.figure(figsize=(15,8))
sns.heatmap(rel.corr(),annot=True)

Dada a matriz de correlação vemos que parâmetros como horas trabalhas por semana, ganho capital, sexo, relacionamento, educação e idade são importantes para o modelo. Parâmetros como fnlwgt pode e vai ser desprezado, education também pois education.num representa a mesma informação e native.country dado que esse parâmetro está enviesado.

Considerando então openas os parâmetros dicutidos e numéricos em sua origem:
Esse classificador foi feito apenas para teste dado que o proximo apresentou uma taxa de acerto maior, este apresentou uma taxa de acerto nos dados de teste de : 82,444%

In [ ]:
trainX=adult_train[["age","education.num","capital.gain", "capital.loss", "hours.per.week"]]
trainY=adult_train.income
testX=adult_test[["age","education.num","capital.gain", "capital.loss", "hours.per.week"]]

Realizando normalização dos dados para eles possuirem escalas parecidas:

In [ ]:
normalizador = preprocessing.StandardScaler()
trainX = normalizador.fit_transform(trainX)
testX = normalizador.fit_transform(testX)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
média=0
maior_média=0
melhor_k=0

Escolhendo o melhor k via cross validation:

In [ ]:
for K in range(1,50):
    knn = KNeighborsClassifier(n_neighbors=K)
    scores = cross_val_score(knn,trainX,trainY,cv=10)
    média=scores.mean()
    if(média>maior_média):
        maior_média=média
        melhor_k=K

O melhor k foi:

In [ ]:
melhor_k


A melhor taxa de erro no cross validation foi:

In [ ]:
maior_média

Construindo o classificador:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=melhor_k)
knn.fit(trainX,trainY)

Testando para o classificador nos dadso de teste:

In [ ]:
testY_res=knn.predict(testX)
testY_res

Criando o arquivo para a submissão desse classificador apenas composto por parâmetros de origem numérica:

In [ ]:
submiss=pd.DataFrame(testY_res,columns=['income'])
submiss.to_csv("PMR3508-2020-90_submissao.csv", index_label="Id")

Agora, irei realizar o classificador com todos os parametros considerados, numericos e não numericos em sua origem. Esse é o classificador final e apresentou uma taxa de erro de : 84.711%

Da mesma forma que no último classificador:

In [ ]:
from sklearn import preprocessing

In [ ]:
trainX = adult_train_num[["age","education.num","capital.gain", "capital.loss", "hours.per.week","occupation",'relationship','race','sex','marital.status']]
trainY=adult_train.income
testX = adult_test_num[["age","education.num","capital.gain", "capital.loss", "hours.per.week","occupation",'relationship','race','sex','marital.status']]

In [ ]:
normalizador = preprocessing.StandardScaler()
trainX = normalizador.fit_transform(trainX)
testX = normalizador.fit_transform(testX)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
média=0
maior_média=0
melhor_k=0
for K in range(1,50):
    knn = KNeighborsClassifier(n_neighbors=K)
    scores = cross_val_score(knn,trainX,trainY,cv=10)
    média=scores.mean()
    if(média>maior_média):
        maior_média=média
        melhor_k=K

In [ ]:
melhor_k

In [ ]:
knn = KNeighborsClassifier(n_neighbors=melhor_k)
knn.fit(trainX,trainY)

In [ ]:
testY_res_num=knn.predict(testX)
testY_res_num

In [ ]:
melhor_k

In [ ]:
maior_média

In [ ]:
submiss=pd.DataFrame(testY_res_num,columns=['income'])
submiss.to_csv("PMR3508-2020-90_submissao.csv", index_label="Id")